In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import MetalDensity
from setups import utilities
from setups import UIdisplay
from setups import FakeTotemGen
from batchSetups import simpleTSFbatch

In [8]:
import importlib
importlib.reload(simpleTSFbatch)
importlib.reload(FakeTotemGen)

confPath = "./conf_UPTR2505/UPTR_p1278_m14_15b15_r1.json"  #"./configures/TSF_org.json"
createFolder = "./UPTR_2505/UPTR_p1278_m14_15b15_r1"

batch = simpleTSFbatch.simpleTFSBatchRun(confPath, createFolder)
batch.genCoreCase()
print(batch.runCMDList)

batch.genSweepCase()

#print(batch.runCMDDict, batch.runCMDList)
batch.runBatch()

./conf_UPTR2505/UPTR_p1278_m14_15b15_r1.json Parsing...
{'//COMMENT1': 'Power: mW, distance: um, HTC: W/(K-m^2), Power_density: W/m^2', 'CORE': {'TOTEMTF': '/nfs/site/disks/x5e2d_gwa_chienchi_001/TechFiles/Totem-EM/fake1278_m14_woBULK.tech', 'LAYERMAP': None, 'THERMALTF': {'PATH': '/nfs/site/disks/x5e2d_gwa_chienchi_001/TechFiles/thermalTF/TF_thermal/Intel/p1278_m14_woBULK.xml', 'NAME': 'p1278_m14_woBULK'}, 'RESOLUTION': 1, 'DESIGN_AREA': {'SIZE': [15, 15]}, 'METAL_DENSITY_MODEL': 'TYPICAL', 'AMBIENT_T': 25, 'HTC': [10000, 1000, 0], 'POWER_CELL0': {'SIZE': [1, 1], 'POWER_VAL': 0.1, 'OFFSET': [7, 7]}}, 'SWEEP': {'UNION': {'POWER_CELL0:OFFSET': [[6, 7], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [0, 7]]}}, 'METAL_DENSITY_LIBRARY': {'TYPICAL': {'METAL': {'0': {'LAYERS': 'GLOBAL', 'DENSITY': 0}}, 'VIA': {'0': {'LAYERS': 'GLOBAL', 'DENSITY': 0}}}}}
/nfs/site/disks/x5e2d_gwa_chienchi_001/TechFiles/Totem-EM/fake1278_m14_woBULK.tech Parsing...
<INFO> Instances Area:7.0,7.0,8.0,8.0; Total PWR=0.10

## Check Submitted Jobs ##

In [11]:
import os 
import importlib

from parsers import RHSCETparser
importlib.reload(RHSCETparser)

caseUnCompleted = []
batchDict = batch.runCMDDict
for case in batchDict.keys():
    if batchDict[case]["status"] in ["DONE"]:
        ETprofilePathList = batchDict[case]["results"]
        for et in ETprofilePathList:
            if not os.path.isfile(et):
                print("<WARNING> Thermal profile: {} not found".format(et))
                caseUnCompleted.append(case)
    else:
        caseUnCompleted.append(case)

print("<INFO> un-completed cases: {}".format(caseUnCompleted))

<INFO> un-completed cases: []


In [10]:
### re-submit un-completed jobs ###
batch.runCases(caseUnCompleted)

<INFO> submit Job: J#S1_P0_0
Your job has been queued (JobID 2191486752, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)
Your job has been queued (JobID 2191486860, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)
<INFO> submit Job: J#S4_P0_3
Your job has been queued (JobID 2191486904, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)
Your job has been queued (JobID 2191487013, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)


## Extract Results ##

In [118]:
import os
import json
import importlib

from parsers import RHSCETparser
importlib.reload(RHSCETparser)

selectedLayer = "NPTAB"
outputFolder = createFolder #"./BatchRunM01"

with open(confPath, "r") as fid:
    inJSON = json.load(fid)

grabPowerCell = False
if "POWER_CELL0" in inJSON["CORE"].keys():
    grabPowerCell = True
    cellDict = inJSON["CORE"]["POWER_CELL0"]
    cellRegion = [cellDict["OFFSET"][0], cellDict["OFFSET"][1],\
                  cellDict["OFFSET"][0]+cellDict["SIZE"][0], cellDict["OFFSET"][1]+cellDict["SIZE"][1]]

cases, TMaxs, TMins = ["Case"], ["Layer TMax"], ["Layer TMin"]
cellTMaxs, cellTAvgs, cellTMins = ["Cell TMax"], ["Cell TAvg"], ["Cell TMin"]
details = []
batchDict = batch.runCMDDict
for case in batchDict.keys():
    caseName = case.split("J#")[-1]
    print("<INFO> Case: {}".format(caseName))
    cases.append(caseName)
    details.append(caseName)
    
    if batchDict[case]["status"] in ["DONE"]:
        ETprofilePathList = batchDict[case]["results"]
        ### only one thermal profile ###
        for et in ETprofilePathList:
            if not os.path.isfile(et):
                print("<WARNING> Thermal profile: {} not found".format(et))
                continue
                
            etView = RHSCETparser.RHSCETView(et, outputFolder=outputFolder)
            TMax = etView.getLayerTMax(selectedLayer=selectedLayer)
            TMin = etView.getLayerTMin(selectedLayer=selectedLayer)
            TMaxs.append(TMax)
            TMins.append(TMin)

            if grabPowerCell:
                cMax, cAvg, cMin, cAll = etView.getLayerRegionalT(selectedLayer=selectedLayer, region=cellRegion)
                cellTMaxs.append(cMax)
                cellTAvgs.append(cAvg)
                cellTMins.append(cMin)
            
            header, vals = etView.getLayerTvals(selectedLayer=selectedLayer)
            details.append(",".join(header))
            details.append(",".join(vals))


if grabPowerCell:
    outStr = [",".join(cases), ",".join(TMaxs), ",".join(TMins), ",".join(cellTMaxs), ",".join(cellTAvgs), ",".join(cellTMins)]
else:
    outStr = [",".join(cases), ",".join(TMaxs), ",".join(TMins)]

reportPath = os.path.join(outputFolder, "report.txt")
with open(reportPath, "w") as fid:
    fid.write("\n".join(outStr))
    fid.write("\n\nDetail\n")
    fid.write("\n".join(details))

print("<INFO> Done")

<INFO> Case: core
<INFO> Case: S1_P0_0
<INFO> Case: S2_P0_1
<INFO> Case: S3_P0_2
<INFO> Case: S4_P0_3
<INFO> Case: S5_P0_4
<INFO> Done


## UT1: Power Cell with Padding Script Generation ##

In [113]:
import os
import json

### template script ###
## [100000,500000,1000000,5000000,10000000,50000000,100000000,200000000,300000000]
runJSON = {
    "//COMMENT1":"Power: mW, distance: um, HTC: W/(K-m^2), Power_density: W/m^2",\
    "CORE": {
      "TOTEMTF": "/nfs/site/disks/ifs_itip_disk001/cchuang/Thermal/RUN_env/RHSC_ET_Batch_v01/DB/tech/fake1P11M_BS4Mi.tech",
      "LAYERMAP": "/nfs/site/disks/ifs_itip_disk001/cchuang/Thermal/RUN_env/RHSC_ET_Batch_v01/DB/tech/layermap_fake_1P11M_BS4Mi.conf",
      "THERMALTF": {
        "PATH": "/nfs/site/disks/ifs_itip_disk001/cchuang/Thermal/RUN_env/RHSC_ET_Batch_v01/DB/thermalTF/withTSV/topTF.xml",
        "NAME": "1P11M_wTSV_BS4Mi"},
      "RESOLUTION": 1,
      "DESIGN_AREA": {
        "SIZE": [100, 100],
        "POWER_SOURCE": {
          "TYPE": "UNIFORM",
          "POWER_VAL": 0
        }
      },
      "METAL_DENSITY_MODEL": "TYPICAL",
      "AMBIENT_T": 25,
      "HTC": [30000, 5000, 0]
    }, 
    "SWEEP": {
      "UNION": {
        "POWER_CELL0:POWER_DENSITY": [10000000,50000000,100000000,200000000,300000000]
      }
    },
    "METAL_DENSITY_LIBRARY": {
      "TYPICAL": {
        "METAL": {"0": {"LAYERS": "GLOBAL", "DENSITY": 0.6}},
        "VIA": {"0": {"LAYERS": "GLOBAL", "DENSITY": 0.1}} 
      }
    }
}

## UT1: Unit Power Source with 0 BG Sweeping Script Generation ##

In [6]:
import os
import json

### template script ###
runJSON = {
    "//COMMENT1":"Power: mW, distance: um, HTC: W/(K-m^2), Power_density: W/m^2",\
    "CORE": {
      "TOTEMTF": "/nfs/site/disks/x5e2d_gwa_chienchi_001/TechFiles/Totem-EM/fake1278_m14_woBULK.tech",
      "LAYERMAP": None,
      "THERMALTF": {
        "PATH": "/nfs/site/disks/x5e2d_gwa_chienchi_001/TechFiles/thermalTF/TF_thermal/Intel/p1278_m14_woBULK.xml",
        "NAME": "p1278_m14_woBULK"
      },
      "RESOLUTION": 1,
      "DESIGN_AREA": {
        "SIZE": [13, 13]
      },
      "METAL_DENSITY_MODEL": "TYPICAL",
      "AMBIENT_T": 25,
      "HTC": [10000, 1000, 0]
    }, 
    "SWEEP": {
      "UNION": {
        "POWER_CELL0:OFFSET": []
      }
    },
    "METAL_DENSITY_LIBRARY": {
      "TYPICAL": {
        "METAL": {"0": {"LAYERS": "GLOBAL", "DENSITY": 0}},
        "VIA": {"0": {"LAYERS": "GLOBAL", "DENSITY": 0}} 
      }
    }
}

In [7]:
### Pad Power Cell setup ###
RESOLUTION = 1            ### unit: 1um
CELL_POWER_VAL = 0.1      ## power for cell in mW
DIE_WIDTH = 15           ## unit: resolution
DIE_HEIGHT = 15
DIE_OFFSETX = 0
DIE_OFFSETY = 0
CELL_WIDTH = 1            ## unit: resolution; 0.05*20=1
CELL_HEIGHT = 1           ## 0.16*20=3.2 -> 3             
CELL_OFFSETX = int((DIE_WIDTH*0.5)-(CELL_WIDTH*0.5)+DIE_OFFSETX)
CELL_OFFSETY = int((DIE_HEIGHT*0.5)-(CELL_HEIGHT*0.5)+DIE_OFFSETY) 

### update runJSON ###
runJSON["CORE"]["DESIGN_AREA"]["SIZE"] = [DIE_WIDTH, DIE_HEIGHT]
runJSON["CORE"]["POWER_CELL0"] = {"SIZE": [CELL_WIDTH, CELL_HEIGHT], "POWER_VAL": CELL_POWER_VAL, "OFFSET": [CELL_OFFSETX, CELL_OFFSETY]}

CELL_MOVE = []
for _x in range(0, CELL_OFFSETX):
    CELL_MOVE.append([CELL_OFFSETX-(_x+1), CELL_OFFSETY])

runJSON["SWEEP"]["UNION"]["POWER_CELL0:OFFSET"] = CELL_MOVE

#print(runJSON)
#save2Folder = "./conf_UPTR"
save2Folder = "./conf_UPTR2505"
fileName = "UPTR_p1278_m14_15b15_r1.json"
if not os.path.isdir(save2Folder):
    os.makedirs(save2Folder, 0o755)

with open(os.path.join(save2Folder, fileName), "w") as fid:
    json.dump(runJSON, fid, indent=2)

print("JSON PATH: {}".format(os.path.join(save2Folder, fileName)))

JSON PATH: ./conf_UPTR2505/UPTR_p1278_m14_15b15_r1.json


In [114]:
### Pad Power Cell setup ###
RESOLUTION = 1            ### unit: 1um
CELL_POWER_DENSITY = 5000000  ## power density for cell
POWER_DENSITY = 5000000   ## power density for background (W/m^2)
DIE_WIDTH = 100           ## unit: resolution
DIE_HEIGHT = 100
DIE_OFFSETX = 0
DIE_OFFSETY = 0
CELL_WIDTH = 2            ## unit: resolution; 0.05*20=1
CELL_HEIGHT = 2           ## 0.16*20=3.2 -> 3             
CELL_OFFSETX = int((DIE_WIDTH*0.5)-(CELL_WIDTH*0.5)+DIE_OFFSETX)
CELL_OFFSETY = int((DIE_HEIGHT*0.5)-(CELL_HEIGHT*0.5)+DIE_OFFSETY)
PADX = 12                 ## unit: resolution;   0.6*20 -> 12, org: 6
PADY = 6                  ## 0.32*20=6.4 -> 6;  0.64*20=12.8 -> 13 

CELL_PADX1 = CELL_OFFSETX-PADX
CELL_PADX2 = CELL_OFFSETX+CELL_WIDTH+PADX
CELL_PADY1 = CELL_OFFSETY-PADY
CELL_PADY2 = CELL_OFFSETY+CELL_HEIGHT+PADY


### update runJSON ###
runJSON["CORE"]["DESIGN_AREA"]["SIZE"] = [DIE_WIDTH, DIE_HEIGHT]
runJSON["CORE"]["POWER_CELL0"] = {"SIZE": [CELL_WIDTH, CELL_HEIGHT], "POWER_DENSITY": CELL_POWER_DENSITY, "OFFSET": [CELL_OFFSETX, CELL_OFFSETY]}
runJSON["CORE"]["POWER_CELL1"] = {"SIZE": [CELL_PADX1, DIE_HEIGHT], "POWER_DENSITY": POWER_DENSITY, "OFFSET": [DIE_OFFSETX, DIE_OFFSETY]}
runJSON["CORE"]["POWER_CELL2"] = {"SIZE": [DIE_OFFSETX+DIE_WIDTH-CELL_PADX2, DIE_HEIGHT], "POWER_DENSITY": POWER_DENSITY, \
                                  "OFFSET": [CELL_PADX2, DIE_OFFSETY]}
runJSON["CORE"]["POWER_CELL3"] = {"SIZE": [PADX+CELL_WIDTH+PADX, CELL_PADY1], "POWER_DENSITY": POWER_DENSITY, "OFFSET": [CELL_PADX1, DIE_OFFSETY]}
runJSON["CORE"]["POWER_CELL4"] = {"SIZE": [PADX+CELL_WIDTH+PADX, DIE_OFFSETY+DIE_HEIGHT-CELL_PADY2], "POWER_DENSITY": POWER_DENSITY, \
                                  "OFFSET": [CELL_PADX1, CELL_PADY2]}

#print(runJSON)
save2Folder = "/nfs/site/disks/ifs_itip_disk001/cchuang/Thermal/RUN_env/RHSC_ET_Batch_v01/conf_UPTR_250326"
fileName = "UPTR_BG5e6_1_100x100_LWS.json"
if not os.path.isdir(save2Folder):
    os.makedirs(save2Folder, 0o755)

with open(os.path.join(save2Folder, fileName), "w") as fid:
    json.dump(runJSON, fid, indent=2)